# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [2]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [3]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [4]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

In [5]:
n_features

296

As a refresher, preview the training data: 

In [6]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1051 entries, 0 to 1050
Columns: 296 entries, MSSubClass to SaleCondition_Partial
dtypes: float64(296)
memory usage: 2.4 MB


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [11]:
# Train the model 
normalized_input_model.fit(X_train,
                           y_train, 
                           batch_size=32,
                           epochs=150,
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [12]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [13]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32,
                     epochs=150,
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5187 - mse: 0.5187 - val_loss: 0.2363 - val_mse: 0.2363
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2595 - mse: 0.2595 - val_loss: 0.1759 - val_mse: 0.1759
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2006 - mse: 0.2006 - val_loss: 0.1567 - val_mse: 0.1567
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1692 - mse: 0.1692 - val_loss: 0.1654 - val_mse: 0.1654
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1519 - mse: 0.1519 - val_loss: 0.1451 - val_mse: 0.1451
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1351 - mse: 0.1351 - val_loss: 0.1380 - val_mse: 0.1380
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1233 - mse: 0.1233 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1106 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1388 - val_mse: 0.1388
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1400 - val_mse: 0.1400
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0195 - mse: 0.0195 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0196 - mse: 0.0196 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0190 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1400 - val_mse: 0.1400
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1401 - val_mse: 0.1401
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1413 - val_mse: 0.1413
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1408 - val_mse: 0.1408
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1396 - val_mse: 0.1396
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [14]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 640us/step - loss: 0.0070 - mse: 0.0070


[0.006956520956009626, 0.006956520956009626]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [15]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 772us/step - loss: 0.1404 - mse: 0.1404


[0.14042699337005615, 0.14042699337005615]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [16]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

29448.066603754934

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, specify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [17]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,),
                          kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5237 - mse: 0.5237 - val_loss: 0.1979 - val_mse: 0.1979
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2595 - mse: 0.2595 - val_loss: 0.1637 - val_mse: 0.1637
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2046 - mse: 0.2046 - val_loss: 0.1470 - val_mse: 0.1470
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1742 - mse: 0.1742 - val_loss: 0.1449 - val_mse: 0.1449
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1568 - mse: 0.1568 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1390 - mse: 0.1390 - val_loss: 0.1368 - val_mse: 0.1368
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1283 - mse: 0.1283 - val_loss: 0.1499 - val_mse: 0.1499
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1201 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0270 - mse: 0.0270 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0259 - mse: 0.0259 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0255 - mse: 0.0255 - val_loss: 0.1201 - val_mse: 0.1201
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1202 - val_mse: 0.1202
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0247 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1190 - val_mse: 0.1190
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1206 - val_mse: 0.1206
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1203 - val_mse: 0.1203
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1199 - val_mse: 0.1199
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 583us/step - loss: 0.0095 - mse: 0.0095


[0.00954829528927803, 0.00954829528927803]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 812us/step - loss: 0.1193 - mse: 0.1193


[0.11929354071617126, 0.11929354071617126]

## Lecun Initialization 

In the cell below, specify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [20]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,),
                             kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4658 - mse: 0.4658 - val_loss: 0.1758 - val_mse: 0.1758
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2204 - mse: 0.2204 - val_loss: 0.1347 - val_mse: 0.1347
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1875 - mse: 0.1875 - val_loss: 0.1221 - val_mse: 0.1221
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1621 - mse: 0.1621 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1446 - mse: 0.1446 - val_loss: 0.1147 - val_mse: 0.1147
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1366 - mse: 0.1366 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1257 - mse: 0.1257 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1152 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1406 - val_mse: 0.1406
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1385 - val_mse: 0.1385
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.1383 - val_mse: 0.1383
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1373 - val_mse: 0.1373
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1374 - val_mse: 0.1374
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1391 - val_mse: 0.1391
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0211 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1416 - val_mse: 0.1416
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1410 - val_mse: 0.1410
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 627us/step - loss: 0.0073 - mse: 0.0073


[0.0072860438376665115, 0.0072860438376665115]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 652us/step - loss: 0.1417 - mse: 0.1417


[0.1417035460472107, 0.1417035460472107]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [24]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
                      loss='mse',
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3273 - mse: 0.3273 - val_loss: 0.1271 - val_mse: 0.1271
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1687 - mse: 0.1687 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1326 - mse: 0.1326 - val_loss: 0.1233 - val_mse: 0.1233
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1100 - mse: 0.1100 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0978 - mse: 0.0978 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0641 - mse: 0.0641 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0592 - mse: 0.0592 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0505 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0087 - mse: 0.0087 - val_loss: 0.1044 - val_mse: 0.1044
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.0978 - val_mse: 0.0978
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.0962 - val_mse: 0.0962
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.1007 - val_mse: 0.1007
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0086 - mse: 0.0086 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0930 - val_mse: 0.0930
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0914 - val_mse: 0.0914
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1037 - val_mse: 0.1037
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 596us/step - loss: 0.0042 - mse: 0.0042


[0.004170762374997139, 0.004170762374997139]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 691us/step - loss: 0.0921 - mse: 0.0921


[0.0921095460653305, 0.0921095460653305]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [27]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                   loss='mse',
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3474 - mse: 0.3474 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1616 - mse: 0.1616 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1151 - mse: 0.1151 - val_loss: 0.1243 - val_mse: 0.1243
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0805 - mse: 0.0805 - val_loss: 0.1036 - val_mse: 0.1036
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0592 - mse: 0.0592 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0496 - mse: 0.0496 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0376 - mse: 0.0376 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0345 - m

Epoch 63/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0041 - mse: 0.0041 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0953 - val_mse: 0.0953
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1016 - val_mse: 0.1016
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.

33/33 [==============================] - 0s 2ms/step - loss: 5.2408e-04 - mse: 5.2408e-04 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 8.1353e-04 - mse: 8.1353e-04 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.0920 - val_mse: 0.0920
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1031 - val_mse: 0.1031
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.0910 - val_mse: 0.0910
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 131/150
33/33 [==============================] - 0s 2ms/step 

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 599us/step - loss: 8.7606e-04 - mse: 8.7606e-04


[0.0008760577766224742, 0.0008760577766224742]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 869us/step - loss: 0.0937 - mse: 0.0937


[0.09371356666088104, 0.09371356666088104]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [30]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 732us/step - loss: 0.1323 - mse: 0.1323


[0.13231196999549866, 0.13231196999549866]

In [31]:
# Adam actually did better with the training data than rmsprop and only very slightly worse on the val data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 740us/step - loss: 0.1232 - mse: 0.1232


[0.12320525199174881, 0.12320525199174881]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [32]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

28584.531523657653

In [33]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

27583.290217994956

In [ ]:
### As shown above, the Adam optimized model actually slightly outperformed the rmsprop optimized 
### model on the test data (lower RMSE).

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.